In [1]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings
import numpy as np
warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('../train/*.csv'))]
from matplotlib import pyplot as plt

In [2]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [3]:
from sklearn.preprocessing import LabelEncoder
years = ['2017', '2018', '2019', '2020']

year_encoder = {}

for year, df in zip(years, know_train):
    print(year)
    encoders = {}
    
    for col in df.columns:
        if col == 'ID':
            continue
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = LabelEncoder()
            df[col] = df[col].map(str)
            df[col] = encoder.fit_transform(df[col])
            encoders[col] = encoder
            
            
    year_encoder[year] = encoders

2017
2018
2019
2020


In [20]:
data = {}
for year, df in zip(years, know_train):
    data[year] = df.drop('idx',axis=1)

In [21]:
bining = {'2017' : ['bq37','bq41_1','bq41_2','bq41_3'] ,
         '2018' : ['bq36','bq41_1','bq41_2','bq41_3'],
         '2019' : ['bq26','bq31_1','bq31_2','bq31_3'],
         '2020' : ['bq25','bq30_1','bq30_2','bq30_3']}
#연령 bining
# 19세 이상 35세이하 '청년'
# 36세이상 65세 미만 '중년'
# 65세 이상 '노년'
for year in years:
    if year == '2017':
        age = 'bq37'
    elif year == '2018':
        age = 'bq36'
    elif year == '2019':
        age = 'bq26'
    else:
        age = 'bq25'
    data[year].loc[(data[year][age] <=35),'young'] =1
    data[year].loc[(data[year][age] > 35),'middle'] =1
    data[year].loc[(data[year][age] >=65),'old'] =1
    data[year].fillna(0,inplace=True)
    #data[year].drop(age,axis=1,inplace=True)

In [22]:
for year in years:
    if year == '2017':
        first = 'bq41_1' 
        second = 'bq41_2'
        thrid = 'bq41_3'
    elif year == '2018':
        first = 'bq41_1'
        second = 'bq41_2'
        thrid = 'bq41_3'
    elif year == '2019':
        first = 'bq31_1'
        second = 'bq31_2'
        thrid = 'bq31_3'
    else:
        first = 'bq30_1'
        second = 'bq30_2'
        thrid = 'bq30_3'
    Q_3 = np.quantile(data[year][first],.75)
    data[year].loc[(data[year][first] <= Q_3 ) , '1LS'] = 1
    data[year].loc[data[year][first] > Q_3 , '1HS'] = 1
    data[year].loc[data[year][first] == 0  , '1NA'] = 1
    
    Q_3 = np.quantile(data[year][second],.75)
    data[year].loc[(data[year][second] <= Q_3 ) , '2LS'] = 1
    data[year].loc[data[year][second] > Q_3 , '2HS'] = 1
    data[year].loc[data[year][second] == 0  , '2NA'] = 1
    
    Q_3 = np.quantile(data[year][thrid],.75)
    data[year].loc[(data[year][thrid] <= Q_3 ) , '3LS'] = 1
    data[year].loc[data[year][thrid] > Q_3 , '3HS'] = 1
    data[year].loc[data[year][thrid] == 0  , '3NA'] = 1
    
   
    
    data[year].fillna(0,inplace=True)
    #data[year].drop([first,second,thrid],axis=1,inplace=True)

In [46]:
train_data = {}
for year in years:
    train_data[year] = {'X': data[year].drop('knowcode',axis=1), # ID제외
                        'y': data[year]['knowcode']} 

In [26]:
from sklearn.preprocessing import StandardScaler
minmax = StandardScaler()

for year in years:
    train_data[year]['X'] = minmax.fit_transform(train_data[year]['X'])
    train_data[year]['X'] = pd.DataFrame(train_data[year]['X'])

In [47]:
train_data['2017']['X']

,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,...,old,1LS,1HS,1NA,2LS,2HS,2NA,3LS,3HS,3NA
0,3,3,3,3,3,3,4,4,3,4,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
1,4,5,4,5,3,4,3,4,3,4,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
2,3,4,3,4,3,4,5,6,4,5,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,3,3,3,3,3,5,4,5,4,6,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,4,5,3,4,3,4,4,5,3,4,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,3,5,2,4,3,3,2,2,2,3,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
9482,5,5,5,5,5,5,3,4,4,5,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
9483,3,3,4,6,3,3,4,5,4,5,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
9484,3,5,3,5,4,5,3,4,3,5,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [45]:
train_data['2017']['X']

,0,1,2,3,4,5,6,7,8,9,...,156,157,158,159,160,161,162,163,164,165
0,-0.224813,-0.534753,-0.091092,-0.434796,-0.101272,-0.457113,1.258696,0.676460,0.367337,0.531336,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
1,0.712582,0.714057,0.837053,0.725558,-0.101272,0.125393,0.457247,0.676460,0.367337,0.531336,...,-0.141809,0.531587,-0.531587,2.474191,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
2,-0.224813,0.089652,-0.091092,0.145381,-0.101272,0.125393,2.060145,1.622418,1.235654,1.030124,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
3,-0.224813,-0.534753,-0.091092,-0.434796,-0.101272,0.707899,1.258696,1.149439,1.235654,1.528912,...,-0.141809,-1.881158,1.881158,-0.404172,0.392089,-0.392089,0.392089,-1.760510,1.760510,-0.401344
4,0.712582,0.714057,-0.091092,0.145381,-0.101272,0.125393,1.258696,1.149439,0.367337,0.531336,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,-0.224813,0.714057,-1.019237,0.145381,-0.101272,-0.457113,-0.344202,-0.269497,-0.500980,0.032548,...,-0.141809,-1.881158,1.881158,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
9482,1.649976,0.714057,1.765197,0.725558,1.778705,0.707899,0.457247,0.676460,1.235654,1.030124,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,-1.760510,1.760510,-0.401344
9483,-0.224813,-0.534753,0.837053,1.305735,-0.101272,-0.457113,1.258696,1.149439,1.235654,1.030124,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
9484,-0.224813,0.714057,-0.091092,0.725558,0.838716,0.707899,0.457247,0.676460,0.367337,1.030124,...,-0.141809,-1.881158,1.881158,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344


In [28]:
from sklearn.model_selection import train_test_split

split_data = {}
for year in tqdm(years):
    X_train,X_test,y_train,y_test = train_test_split(train_data[year]['X'],train_data[year]['y'],test_size=0.25, random_state=42)
    split_data[year] = {'X_train' : X_train,
                       'y_train' : y_train,
                       'X_test' : X_test,
                       'y_test' : y_test}

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 65.36it/s]


In [44]:
split_data['2017']['X_train']

,0,1,2,3,4,5,6,7,8,9,...,156,157,158,159,160,161,162,163,164,165
6263,-0.224813,0.089652,0.837053,0.725558,0.838716,0.707899,-0.344202,-0.269497,0.367337,0.531336,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
8676,0.712582,0.714057,0.837053,0.725558,-0.101272,0.707899,-1.145652,-1.215455,-1.369298,-1.463816,...,-0.141809,0.531587,-0.531587,2.474191,-2.550441,2.550441,-2.550441,0.568017,-0.568017,2.491627
4844,1.649976,0.714057,0.837053,1.305735,0.838716,0.707899,-0.344202,-0.742476,-0.500980,-0.466240,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,-1.760510,1.760510,-0.401344
2492,-0.224813,-0.534753,-1.947382,-2.175327,-1.981249,-2.204633,-0.344202,0.203482,-1.369298,-1.463816,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
8214,-2.099601,-2.407969,-1.947382,-2.175327,-1.981249,-2.204633,-1.145652,-1.215455,-1.369298,-1.463816,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,0.712582,1.338463,0.837053,1.305735,0.838716,1.290406,-1.145652,-1.215455,1.235654,1.030124,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
5191,0.712582,0.714057,0.837053,0.725558,0.838716,0.707899,-1.145652,-1.215455,1.235654,1.030124,...,-0.141809,-1.881158,1.881158,-0.404172,0.392089,-0.392089,0.392089,-1.760510,1.760510,-0.401344
5390,0.712582,1.338463,1.765197,1.305735,0.838716,0.707899,0.457247,0.676460,1.235654,1.030124,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344
860,0.712582,0.714057,1.765197,1.305735,0.838716,1.290406,0.457247,-0.742476,0.367337,0.531336,...,-0.141809,0.531587,-0.531587,-0.404172,0.392089,-0.392089,0.392089,0.568017,-0.568017,-0.401344


In [41]:
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

model = [SVC(random_state=42),
         xgb.XGBClassifier(objective = 'multi : softmax',eval_metric = 'mlogloss',random_state=42),
         LogisticRegression(random_state=42),
         RandomForestClassifier(random_state=42),
         LGBMClassifier(objective = 'multiclass',metric ='multi_logloss',random_state=42)]
model_f1 = []
f1 = []
for clf in model:
    models ={}
    for year in tqdm(years):
        clf.fit(split_data[year]['X_train'], split_data[year]['y_train'])
        y_pred = clf.predict(split_data[year]['X_test'])
        model_f1.append(f1_score(split_data[year]['y_test'],y_pred,average ='micro'))
    f1.append(np.mean(model_f1))

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:43<00:00, 55.86s/it]


In [42]:
f1

[0.3147369789342521,
 0.42514602299667836,
 0.4250931910991931,
 0.44914798573404585,
 0.37348849488039615]